## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-15-23-52-07 +0000,nypost,YouTubers accused of swiping donations from We...,https://nypost.com/2025/11/15/us-news/youtuber...
1,2025-11-15-23-27-53 +0000,nypost,Sen. John Fetterman shares gnarly photo of blo...,https://nypost.com/2025/11/15/us-news/john-fet...
2,2025-11-15-23-12-47 +0000,latimes,"5-year-old girl missing, father dead after lar...",https://www.latimes.com/california/story/2025-...
3,2025-11-15-23-12-20 +0000,bbc,Thousands protest against government in Mexico...,https://www.bbc.com/news/articles/cn8vm30rr78o...
4,2025-11-15-23-08-05 +0000,nypost,Video shows daring helicopter rescue of BASE j...,https://nypost.com/2025/11/15/us-news/video-sh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,42
93,epstein,12
17,he,10
124,former,7
166,taylor,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,112
175,2025-11-15-01-02-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...,103
84,2025-11-15-15-47-56 +0000,nypost,Trump calls Marjorie Taylor Greene ‘disgrace’ ...,https://nypost.com/2025/11/15/us-news/trump-ca...,91
98,2025-11-15-13-22-06 +0000,cbc,Trump calls it quits with Marjorie Taylor Gree...,https://www.cbc.ca/news/world/u-s-trump-greene...,89
150,2025-11-15-03-59-10 +0000,nypost,Trump withdraws endorsement of Rep. Marjorie T...,https://nypost.com/2025/11/14/us-news/trump-wi...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,112,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...
61,33,2025-11-15-17-54-19 +0000,wapo,Former Fed governor violated central bank’s tr...,https://www.washingtonpost.com/business/2025/1...
77,31,2025-11-15-16-45-59 +0000,nypost,Miss Israel getting death threats after being ...,https://nypost.com/2025/11/15/world-news/miss-...
79,29,2025-11-15-16-28-51 +0000,nyt,Fears Fuel Tanker Was Seized by Iran in Strait...,https://www.nytimes.com/2025/11/14/world/middl...
119,28,2025-11-15-10-56-49 +0000,cbc,Vatican returns Indigenous cultural items to C...,https://www.cbc.ca/news/vatican-returns-indige...
137,28,2025-11-15-10-00-00 +0000,wsj,Trump administration officials including Healt...,https://www.wsj.com/politics/policy/rfk-jr-fda...
146,24,2025-11-15-07-21-56 +0000,wapo,"ESPN blackout on YouTube TV ends, as Disney an...",https://www.washingtonpost.com/business/2025/1...
179,23,2025-11-15-00-36-28 +0000,cbc,What will the federal government's Major Proje...,https://www.cbc.ca/news/politics/major-project...
96,23,2025-11-15-13-41-18 +0000,nypost,Let our people go! Zohran Mamdani victory spur...,https://nypost.com/2025/11/15/us-news/staten-i...
31,22,2025-11-15-20-16-50 +0000,latimes,4 dead after panga boat capsizes off Southern ...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
